In [1]:
# HIDDEN
from ipywidgets import widgets, interact, Dropdown,SelectMultiple, Checkbox,VBox,FloatSlider,Button
from IPython.display import display
#https://stackoverflow.com/questions/18398929/passing-multiple-variables-from-dropdowns-using-django-python
#https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
#https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e
#https://stackoverflow.com/questions/26352555/how-does-one-get-widget-values-with-a-button-in-ipython
#https://elc.github.io/posts/embed-interactive-notebooks/

In [2]:
# HIDDEN 
import numpy as np
import pandas as pd
import re
import os
from IPython import get_ipython
ipython = get_ipython()

In [3]:
# HIDDEN
style = {'description_width': 'initial'}
mnths = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
cntry = np.load('trandict.npy',allow_pickle='TRUE').item()
cntry = {v:k for k,v in cntry.items()}

# Choose your country and the month for which you'd like a diet

In [4]:
# HIDDEN
var1 = widgets.Dropdown(options = cntry.keys(),value = 'Switzerland')
display(var1)
var2 = widgets.Dropdown(options = mnths.keys(),value='aug')
display(var2)

Dropdown(index=26, options=('Aruba', 'Angola', 'Anguilla', 'Albania', 'Andorra', 'Argentina', 'American Samoa'…

Dropdown(index=7, options=('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')…

In [5]:
runcode2 = widgets.Button(description='Build the database')
output2 = widgets.Output()
def test(b):
    global dftest2
    with output2:
        print ('I want a diet for:',var1.value, '\nfor the month of:',var2.value)
        ipython.magic('run Build_Optimization_Database.ipynb')
        dftest2 = calc_impacts(cntry[var1.value],var2.value)
        return dftest2

In [6]:
display(runcode2,output2)
runcode2.on_click(test)

Button(description='Build the database', style=ButtonStyle())

Output()

In [7]:
runcode10 = widgets.Button(description='Show the database')
output10 = widgets.Output()
def show(b):
    global dftest2
    with output10:
        print (dftest2.head(1))

In [8]:
display(runcode10,output10)
runcode10.on_click(show)

Button(description='Show the database', style=ButtonStyle())

Output()

In [9]:
# HIDDEN
gender={'female':1,'male':0}
preferences = {'vegetarian':0,'vegan':1,'none':2}
indicator = {'climate change':'GHG','biodiversity loss':'BIO','both':'any'}
activity = {'sedentary':0,'moderate':1,'vigorous':2}
supp = {'yes':'yes_sup','no':'no_sup'}
weighting = {'climate change':0,'biodiversity loss':0,'both':0.5}
def weighting(impact):
    if impact == 'GHG' or 'BIO': w = 0
    elif impact == 'both':w = 0.5
    else: w = 0
    return w

# Please answer the following questions to personalize the diet to your needs

In [10]:
#HIDDEN
imp = widgets.Dropdown(options=indicator.values(),description='Impact')
display(imp)
gen = widgets.Dropdown(options=gender.values(),description='Gender:')
display(gen)
weight = widgets.IntText(description='Weight (kg):',value=60)
display(weight)
height = widgets.IntText(description='Height (cm):',value=165)
display(height)
age = widgets.IntText(description='Age (yrs):',value=40)
display(age)
act = widgets.Dropdown(options=activity.keys(),description='Activity Level:',value='moderate')
display(act)
sup = widgets.Dropdown(options=supp.values(),description='Supplements?',style=style)
display(sup)
days = widgets.BoundedIntText(description='Number of Days:',style=style,value=7)
display(days)
diettype = widgets.Dropdown(options=preferences.keys(),description='Dietary Preferences:',style=style,value='none')
display(diettype)


Dropdown(description='Impact', options=('GHG', 'BIO', 'any'), value='GHG')

Dropdown(description='Gender:', options=(1, 0), value=1)

IntText(value=60, description='Weight (kg):')

IntText(value=165, description='Height (cm):')

IntText(value=40, description='Age (yrs):')

Dropdown(description='Activity Level:', index=1, options=('sedentary', 'moderate', 'vigorous'), value='moderat…

Dropdown(description='Supplements?', options=('yes_sup', 'no_sup'), style=DescriptionStyle(description_width='…

BoundedIntText(value=7, description='Number of Days:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Dietary Preferences:', index=2, options=('vegetarian', 'vegan', 'none'), style=Descripti…

In [12]:
runcode5 = widgets.Button(description='Run Tool')
output5 = widgets.Output()

In [23]:
def run_tool(b):
    #global df
    with output5:
        ipython.magic('run Optimization_Tool_Online.ipynb')
        df_foods = runopt(dftest2,imp.value,diettype.value,sup.value,gen.value,weight.value,\
                          height.value,age.value,act.value,days.value,2)
        return df_foods

In [ ]:
display(runcode5,output5)
runcode5.on_click(run_tool)

In [ ]:
# HIDDEN
def multi_checkbox_widget(descriptions):
    """ Widget with a search field and lots of checkboxes """
    #search_widget = widgets.Text()
    options_dict = {description: widgets.Checkbox(description=description, value=False) for 
                    description in descriptions}
    options = [options_dict[description] for description in descriptions]
    options_widget = widgets.VBox(options, layout={'overflow': 'scroll'})
    #options_widget = widgets.HBox(options, layout={'width': 'max-content'})
    #multi_select = widgets.VBox([search_widget, options_widget])
    mass_dict = {description: widgets.IntSlider(description='Mass (grams)', min=0,
                max = 1000,step=50,value=0,disabled=False) for description in descriptions}
    mass = [mass_dict[description] for description in descriptions]
    mass_widget = widgets.VBox(mass, layout={'overflow': 'scroll'})
    multi_select = widgets.HBox(children = [options_widget,mass_widget], layout={'overflow': 'scroll'})
    #multi_select = widgets.VBox([options_widget])

    # Wire the search field to the checkboxes
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = [options_dict[description] for description in descriptions]
        else:
            # Filter by search field using difflib.
            close_matches = difflib.get_close_matches(search_input, descriptions, cutoff=0.0)
            new_options = [options_dict[description] for description in close_matches]
        options_widget.children = new_options

    #search_widget.observe(on_text_change, names='value')
    return multi_select
# words = list(set([i.split(',')[0] for i in df1[df1['Group']=='FAT']['Food Name'].tolist()]))
# widget = multi_checkbox_widget(words)
# [widget.description for widget in widget.children[1].children if widget.value]

In [ ]:
# HIDDEN
def joinlists(widgetname):
    rs1 = [widgetname.description for widgetname in widgetname.children[0].children if widgetname.value]
    rs2 = [widgetname.value for widgetname in widgetname.children[1].children if widgetname.value]
    food = list(zip(rs1,rs2))
    return food

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['vegetarianproducts'])]['Food Name']]
widget1 = multi_checkbox_widget(words)
widget1

In [ ]:
listofcheckboxes = {'fruits':['FAT'],'vegetables':['DGR','DGC'],
     'legumes':['DBR','DBC','DFR','DFC'],'grains':['AC','AC_wg','DAR','DAC',
     'AI','AI_wg','AA','AA_wg','AF','AF_wg','AD','AD_wg','AC','AC_wg'],'nuts_seeds':
     ['GAT'],'milk':['BAE','BAH','BAK','BLS','BLM','BLH','BLF','BN'],
     'vegetarianproducts':['BAV','BNV','VEG'],
     'fish':['JAC', 'JAR', 'JCC', 'JCR', 'JKC', 'JKR', 'JMC', 'JMR', 'JRC'],
    'meat':['MBGR','MBGC','MACR','MACC','MAER','MECC','MECR','MEER','MEEC','MAIC','MAIR','MAE',\
    'MAIR','MAIC','MCOR','MCOC','MCAR','MCAC','MAGR','MAGC','MAHR','MAHC',
    'MI','MAAC','MAAR'],'eggs_oil':['CA','OA', 'OC', 'OE','BTM','BJC']}

In [ ]:
vegetarianproducts = joinlists(widget1)

In [ ]:
#vegetarianproducts = [widget.description for widget in widget.children[0].children if widget.value]

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['eggs_oil'])]['Food Name']]
widget2 = multi_checkbox_widget(words)
widget2

In [ ]:
eggs_oil = joinlists(widget2)

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['fish'])]['Food Name']]
widget3 = multi_checkbox_widget(words)
widget3

In [ ]:
fish = joinlists(widget3)

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['meat'])]['Food Name']]
widget4 = multi_checkbox_widget(words)
widget4

In [ ]:
meat = joinlists(widget4)

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['fruits'])]['Food Name']]
widget5 = multi_checkbox_widget(words)
widget5

In [ ]:
fruit = joinlists(widget5)

In [ ]:
fruit

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['vegetables'])]['Food Name']]
widget6 = multi_checkbox_widget(words)
widget6

In [ ]:
vegetables = joinlists(widget6)

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['legumes'])]['Food Name']]
widget7 = multi_checkbox_widget(words)
widget7

In [ ]:
legumes = joinlists(widget7)

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['grains'])]['Food Name']]
widget8 = multi_checkbox_widget(words)
widget8

In [ ]:
grains = joinlists(widget8)

In [ ]:
words = [i for i in df1[df1['Group'].isin(listofcheckboxes['milk'])]['Food Name']]
widget9 = multi_checkbox_widget(words)
widget9

In [ ]:
milk = joinlists(widget9)

In [ ]:
selectedfoods = {}
for i in [vegetarianproducts,eggs_oil,fish,meat,fruit,vegetables,legumes,grains,milk]:
    for j in i:
        selectedfoods[j[0].split(',')[0]] = {}#'names'
        selectedfoods[j[0].split(',')[0]]['names']=j[0]
        selectedfoods[j[0].split(',')[0]]['mass']=j[1]

In [ ]:
selectedfoods